In [16]:
import pandas as pd

# Read the csv file
df = pd.read_csv('D:\\Articles\\Code Smells\\Sample Code and Dataset\\VIP - MLCQ Industry-relevant code smell data set\\MLCQCodeSmellSamples.csv')

# Print it out if you want
print(df.shape, "\n")
print("This dataset consists of following features:\n", df.columns) # getting the features' name

(14853, 15) 

This dataset consists of following features:
 Index(['id', 'reviewer_id', 'sample_id', 'smell', 'severity',
       'review_timestamp', 'type', 'code_name', 'repository', 'commit_hash',
       'path', 'start_line', 'end_line', 'link',
       'is_from_industry_relevant_project'],
      dtype='object')


In the next we're going to delete the rows containing none value for severity. These rows are useless in our computations.

In [20]:
dfWithoutNone = df[df['severity'] != "none"]
dfWithoutNone.shape
dfWithoutNone.to_csv("D:\\Articles\\Code Smells\\CodeSmells\\Smell Project\\DatasetWithoutNone.csv")